## File Description
This explores an informed search strategy. In particular, Greedy Best-First Search, A* Search and Iterative Deepening A* Search will be examined with source code.

In [ ]:
import heapq

## Greedy Best-First Search

### 1. Process
*    Initialisation ('priority' queue, visited set)
*    Main loop
    1. Choose one node from priority queue (i.e., choose the smallest value of f(n))
    2. Check if the node is goal (if yes, call the function that output the path from an initial node to a goal node.)
    3. Check if the node is visited previously (if yes, it will be skipped. if not, the node will be added in the visited set).
    4. Expand the next nodes (i.e., the current node's children).

### 2. Advantages
*    It does not require more memory than A* Search

### 3. Disadvantages
*    Imcomplete
*    Not optimal solution

In [ ]:
def greedy_best_first_search(tree, start, goal, heuristic):
    """
    Performs Greedy Best-First Search on a tree/graph.
    :param tree: A dictionary representing the tree/graph as an adjacency list.
    :param start: The starting node.
    :param goal: The goal node.
    :param heuristic: A dictionary mapping nodes to their heuristic values.
    :return: The path from start to goal if found, or None if not found.
    """
    # Initialise the priority queue with the start node
    frontier = [(heuristic[start], start)]  # (heuristic value, node)
    heapq.heapify(frontier)

    visited = set()  # A set to track visited nodes
    parent = {}  # To reconstruct the path

    while frontier:
        # Extract the node with the smallest heuristic value
        _, current = heapq.heappop(frontier)

        # Check if the current node is the goal
        if current == goal:
            return reconstruct_path(parent, start, goal)

        # Skip if the node has already been visited
        if current in visited:
            continue

        # Mark the current node as visited
        visited.add(current)

        # Expand the node's children
        for child in tree.get(current, []):
            if child not in visited:
                heapq.heappush(frontier, (heuristic[child], child))
                parent[child] = current

    # Return None if no path to the goal was found
    return None

def reconstruct_path(parent, start, goal):
    """
    Reconstructs the path from start to goal using the parent dictionary.
    :param parent: A dictionary mapping each node to its parent.
    :param start: The starting node.
    :param goal: The goal node.
    :return: A list representing the path from start to goal.
    """
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = parent.get(current)
    path.append(start)
    path.reverse()
    return path

In [ ]:
# Example tree represented as an adjacency list
example_tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': [],
    'E': [],
    'F': [],
    'G': []
}

# Example heuristic values (lower is better)
example_heuristic = {
    'A': 7,
    'B': 6,
    'C': 4,
    'D': 5,
    'E': 2,
    'F': 1,
    'G': 3
}

# Define the start and goal nodes
start_node = 'A'
goal_node = 'F'

# Run the Greedy Best-First Search
result = greedy_best_first_search(example_tree, start_node, goal_node, example_heuristic)

# Display the result
print(f"Path from {start_node} to {goal_node}: {result}")

Path from A to F: ['A', 'C', 'F']


## A* Search

### 1. Process
*    Initialisation (priority queue, visited set, and **set for storing g(n)**)
*    Main loop
    1. Choose one node from priority queue (i.e., choose the smallest value of f(n) **including g(n)**.)
    2. Check if the node is goal (if yes, call the function that output the path from an initial node to a goal node.)
    3. Check if the node is visited previously (if yes, it will be skipped. if not, the node will be added in the visited set).
    4. Expand the next nodes (i.e., the current node's children). Meanwile, the algorithm calculate g(n) to update a priority queue to be able to find a best choice.

### 2. Advantages
*    Completeness
*    Optimal solution

### 3. Disadvantages
*    It requires more memory than Greedy Best-First Search since it calculate g(n) at each parent.

In [ ]:
import heapq

def a_star_search(tree, start, goal, heuristic, edge_costs):
    """
    Performs A* search on a tree/graph.
    :param tree: A dictionary representing the graph as an adjacency list.
    :param start: The starting node.
    :param goal: The goal node.
    :param heuristic: A dictionary mapping nodes to heuristic values.
    :param edge_costs: A dictionary mapping (parent, child) pairs to edge costs.
    :return: The path from start to goal if found, or None if no path exists.
    """
    # Priority queue (min-heap) with (f(n), node)
    frontier = [(0 + heuristic[start], start)]  # (f(n), node)
    heapq.heapify(frontier)

    g_score = {start: 0}  # Cost from start to each node
    parent = {}  # To reconstruct the path
    visited = set()  # To keep track of visited nodes

    while frontier:
        # Extract the node with the smallest f(n)
        _, current = heapq.heappop(frontier)

        # If the current node is the goal, reconstruct and return the path
        if current == goal:
            return reconstruct_path(parent, start, goal)

        # Mark the current node as visited
        visited.add(current)

        # Expand the node's neighbours
        for neighbour in tree.get(current, []):
            if neighbour in visited:
                continue

            # Calculate the tentative g(n)
            tentative_g = g_score[current] + edge_costs.get((current, neighbour), float('inf'))

            # If the tentative g(n) is better, update the frontier and scores
            if neighbour not in g_score or tentative_g < g_score[neighbour]:
                g_score[neighbour] = tentative_g
                f_score = tentative_g + heuristic[neighbour]
                heapq.heappush(frontier, (f_score, neighbour))
                parent[neighbour] = current

    # Return None if no path is found
    return None


def reconstruct_path(parent, start, goal):
    """
    Reconstructs the path from start to goal using the parent dictionary.
    :param parent: A dictionary mapping each node to its parent.
    :param start: The starting node.
    :param goal: The goal node.
    :return: A list representing the path from start to goal.
    """
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = parent.get(current)
    path.append(start)
    path.reverse()
    return path


In [ ]:
# Example graph as an adjacency list
example_tree = {
    'A': ['B', 'C'],
    'B': ['D', 'E'],
    'C': ['F', 'G'],
    'D': [],
    'E': [],
    'F': [],
    'G': []
}

# Example heuristic values (lower values are better)
example_heuristic = {
    'A': 7,
    'B': 6,
    'C': 4,
    'D': 5,
    'E': 2,
    'F': 1,
    'G': 3
}

# Example edge costs
example_edge_costs = {
    ('A', 'B'): 1,
    ('A', 'C'): 2,
    ('B', 'D'): 4,
    ('B', 'E'): 1,
    ('C', 'F'): 3,
    ('C', 'G'): 1
}

# Define the start and goal nodes
start_node = 'A'
goal_node = 'F'

# Run A* search
result = a_star_search(example_tree, start_node, goal_node, example_heuristic, example_edge_costs)

# Display the result
print(f"Path from {start_node} to {goal_node}: {result}")


Path from A to F: ['A', 'C', 'F']


## Iterative Deepening A* Search

### 1. Process
* Initialisation
  *    Use stack (LIFO)
  *    Prepare the set for calluculating minimum f-score if the predetermined threshold is smaller than the f-score.
*    Main loop starts
  *    Calculate f(n) = g(n) + h(n)
  *    Check if f(n) is larger than the predetermined threshold.
    *    => If yes, the minimum of all f(n) over the threshold is selected as a next threshold.
  *    Check if current node is goal
    *    => If yes, return
  *    Expand next children and assign them in the frontier(stack)
  *    loop will be finished when the goal is found or no space can be explored.


### 2. Advantages
*    Completeness
*    Optimal solution
*    Less memory than A* search

### 3. Disadvantages
*    More time than A* search

In [1]:
import math

def ida_star_search(tree, start, goal, heuristic, edge_costs):
    """
    Performs Iterative Deepening A* (IDA*) search on a tree/graph.
    :param tree: A dictionary representing the graph as an adjacency list.
    :param start: The starting node.
    :param goal: The goal node.
    :param heuristic: A dictionary mapping nodes to heuristic values.
    :param edge_costs: A dictionary mapping (parent, child) pairs to edge costs.
    :return: The path from start to goal if found, or None if no path exists.
    """
    # Initial threshold is set to the heuristic value of the start node
    threshold = heuristic[start]

    # This loop performs iterative deepening over increasing thresholds
    while True:
        # Stack to manage nodes during depth-limited search (acting as the frontier)
        stack = [(start, 0)]  # (node, cumulative cost g(n))
        parent = {}  # Tracks the parent of each node for path reconstruction
        min_threshold = math.inf  # Smallest f(n) exceeding the current threshold

        # Perform depth-limited search
        while stack:
            # Retrieve the current node and its cumulative cost
            current, g_score = stack.pop()

            # Calculate f(n) for the current node
            f_score = g_score + heuristic[current]

            # If f(n) exceeds the threshold, update the smallest exceeding threshold
            if f_score > threshold:
                min_threshold = min(min_threshold, f_score)
                continue

            # If the goal is reached, reconstruct and return the path
            if current == goal:
                return reconstruct_path(parent, start, goal)

            # Explore neighbours (children of the current node)
            for neighbour in tree.get(current, []):
                # Calculate the edge cost to the neighbour
                edge_cost = edge_costs.get((current, neighbour), float('inf'))
                new_g_score = g_score + edge_cost

                # Add the neighbour to the stack if it has not been visited in this path
                if neighbour not in parent or new_g_score < g_score:
                    parent[neighbour] = current
                    stack.append((neighbour, new_g_score))

        # If no path was found, update the threshold and try again
        if min_threshold == math.inf:
            return None  # No solution exists
        threshold = min_threshold


def reconstruct_path(parent, start, goal):
    """
    Reconstructs the path from start to goal using the parent dictionary.
    :param parent: A dictionary mapping each node to its parent.
    :param start: The starting node.
    :param goal: The goal node.
    :return: A list representing the path from start to goal.
    """
    path = []
    current = goal
    while current != start:
        path.append(current)
        current = parent[current]  # Move backwards through the parent mapping
    path.append(start)
    path.reverse()  # Reverse the path to start at the beginning
    return path


In [3]:
# Define the graph as an adjacency list
tree = {
    "A": ["B", "C"],
    "B": ["D", "E"],
    "C": ["F"],
    "D": [],
    "E": ["G"],
    "F": ["G"],
    "G": []
}

# Define heuristic values for each node (Note: All heuristics can not be overestimated due to the nature of admissibility)
heuristic = {
    "A": 7,
    "B": 6,
    "C": 5,
    "D": 4,
    "E": 3,
    "F": 4,
    "G": 0
}

# Define edge costs between nodes
edge_costs = {
    ("A", "B"): 1,
    ("A", "C"): 4,
    ("B", "D"): 5,
    ("B", "E"): 2,
    ("C", "F"): 2,
    ("E", "G"): 5,
    ("F", "G"): 1
}

# Run the IDA* search algorithm
solution_path = ida_star_search(tree, "A", "G", heuristic, edge_costs)
print("Solution Path:", solution_path)

Solution Path: ['A', 'B', 'E', 'G']


## EOF